In [1]:
import pdblp
from xbbg import blp
import pandas as pd
import workdays
import datetime
import numpy as np

con = pdblp.BCon(timeout=10000)
con.start()

C = ["US","DM","UK","CD","AD","NZ","SZ","NK","SK"]

T0 = ["L","YC"]
tmp = ["Currency","country_iso","px_last","volatility_90d","rsi_30d"]

for i in T0:
    T = ["SB" + x + "13" + i + " Index" for x in C]
    BDP0 = blp.bdp(tickers=T, flds=tmp)
    BDP0 = BDP0.loc[T,:]

    for j in tmp[2:]:
        BDP0[j.lower()] = round(BDP0[j.lower()].astype(float),2)
    
    BDP0.insert(1, "Ticker", T)
    BDP0 = BDP0.reset_index(drop=True)
    
    exec(f"T{i} = T")
    exec(f"BDP{i} = pd.DataFrame(BDP0.values, columns = BDP0.columns)")

    
BDPYC["currency"] = BDPL["currency"] 
BDPYC["country_iso"] = BDPL["country_iso"] 

BDPL

,currency,Ticker,country_iso,px_last,volatility_90d,rsi_30d
0,USD,SBUS13L Index,US,515.62,3.54,49.13
1,EUR,SBDM13L Index,DE,364.83,2.86,49.43
2,GBP,SBUK13L Index,GB,694.67,2.62,40.3
3,CAD,SBCD13L Index,CA,644.6,2.71,42.42
4,AUD,SBAD13L Index,AU,1147.82,2.58,34.97
5,NZD,SBNZ13L Index,NZ,409.65,2.0,48.8
6,CHF,SBSZ13L Index,CH,228.86,2.17,48.06
7,NOK,SBNK13L Index,NO,233.71,2.02,44.71
8,SEK,SBSK13L Index,SE,353.52,2.72,50.77


In [2]:
BDPYC

,currency,Ticker,country_iso,px_last,volatility_90d,rsi_30d
0,USD,SBUS13YC Index,US,209.47,3.49,41.45
1,EUR,SBDM13YC Index,DE,194.39,2.8,43.3
2,GBP,SBUK13YC Index,GB,167.5,2.59,33.11
3,CAD,SBCD13YC Index,CA,211.84,2.67,35.55
4,AUD,SBAD13YC Index,AU,176.03,2.56,29.17
5,NZD,SBNZ13YC Index,NZ,104.95,2.0,37.62
6,CHF,SBSZ13YC Index,CH,183.8,2.16,45.15
7,NOK,SBNK13YC Index,NO,103.44,1.96,36.55
8,SEK,SBSK13YC Index,SE,148.51,2.67,43.09


In [3]:
d_from = "20040901"
d_to = workdays.workday(datetime.datetime.today(), days=-1).strftime("%Y%m%d")
Currency = list(BDPYC['currency'])

for i in T0:
    
    exec(f"T = T{i}")
    tmp = con.bdh(T, "px_last", d_from, d_to,
                 elms = [("nonTradingDayFillOption","NON_TRADING_WEEKDAYS"),("nonTradingDayFillMethod","PREVIOUS_VALUE")]).reset_index()

    GBI = pd.DataFrame(np.zeros([len(tmp),len(T)+1]), columns = ["Date"]+T)
    GBI["Date"] = tmp["date"]

    for j in T:
        GBI[j] = tmp[j].values
        
    exec(f"GBI{i} = pd.DataFrame(GBI.values, columns = ['Date'] + Currency)")

GBIL

,Date,USD,EUR,GBP,CAD,AUD,NZD,CHF,NOK,SEK
0,2004-09-01,376.1016,307.28,486.1468,453.63,617.38,222.56,214.2044,163.8212,277.9606
1,2004-09-02,375.7609,307.1,485.9867,453.26,617.8,222.74,214.0897,163.6543,277.9814
2,2004-09-03,374.8351,306.47,485.475,452.54,617.61,222.68,213.7766,163.3528,277.4152
3,2004-09-06,374.9337,306.6,486.095,452.74,616.39,222.56,213.8509,163.587,277.6091
4,2004-09-07,375.1155,306.43,485.886,452.64,616.6,222.58,213.9253,163.6182,277.4953
...,...,...,...,...,...,...,...,...,...,...
4892,2023-06-02,515.3988,365.4,696.3068,647.01,1156.61,411.06,229.2351,234.4759,353.6994
4893,2023-06-05,515.7119,364.91,695.2256,646.75,1154.15,411.12,229.2639,234.1655,353.4954
4894,2023-06-06,515.4428,365.03,694.942,645.99,1152.63,410.8,229.2518,234.0676,353.5202
4895,2023-06-07,515.1169,364.63,694.1531,643.74,1151.3,410.5,229.1078,234.0145,353.3166


In [4]:
HC = pd.DataFrame(np.zeros([len(GBIL),GBIL.shape[1]+1]),columns=["Date","ME"]+list(GBIL.columns[1:]))

HC["Date"] = GBIL["Date"]
for i in range(1,len(HC)):
    if HC.loc[i,"Date"].month != HC.loc[i-1,"Date"].month:
        if HC.loc[i-1,"Date"].month ==3:
            HC.loc[i-1,"ME"] = 3
        else:
            HC.loc[i-1,"ME"] = 1
HC.loc[len(HC)-1,"ME"] = 3

for i in Currency:
    HC.loc[1:,i] = GBIYC[i].pct_change() - GBIL[i].pct_change() 
  
for i in Currency:
    HC[i+"_Index"] = 100
    HC.loc[1:,i+"_Index"] = (1+HC.loc[1:,i]).cumprod()*100
    
HCM = HC.loc[HC["ME"]>=3,["Date"]+[x + "_Index" for x in Currency]]

for i in Currency:
    HCM.loc[1:,i] = HCM.loc[1:,i+"_Index"].pct_change()

HCM = HCM.iloc[1:].reset_index(drop=True)
HCM = HCM[["Date"]+Currency]
for i in range(len(HCM)):
    HCM.iloc[i,1:] = HCM.iloc[i,1:].apply("{:.1%}".format)
HCM

,Date,USD,EUR,GBP,CAD,AUD,NZD,CHF,NOK,SEK
0,2006-03-31,-3.8%,-2.3%,-4.5%,-3.0%,-5.5%,-7.0%,-0.8%,-1.8%,-1.8%
1,2007-03-30,-4.9%,-2.9%,-4.5%,-3.9%,-5.6%,-7.0%,-1.3%,-2.5%,-2.3%
2,2008-03-31,-4.4%,-3.5%,-5.2%,-3.9%,-6.0%,-7.7%,-1.8%,-4.4%,-3.3%
3,2009-03-31,-2.1%,-3.1%,-3.9%,-2.5%,-6.2%,-4.6%,-1.2%,-5.2%,-3.5%
4,2010-03-31,-0.4%,-0.4%,-0.5%,-0.4%,-3.5%,-2.8%,0.0%,-1.5%,-0.2%
5,2011-03-31,-0.4%,-0.6%,-0.6%,-0.9%,-4.8%,-3.1%,0.0%,-2.3%,-1.1%
6,2012-03-30,-0.4%,-0.9%,-0.8%,-1.3%,-5.0%,-2.9%,0.1%,-2.4%,-2.4%
7,2013-03-29,-0.3%,-0.1%,-0.5%,-1.1%,-3.7%,-2.9%,0.2%,-1.7%,-1.6%
8,2014-03-31,-0.2%,-0.1%,-0.5%,-1.1%,-2.7%,-2.7%,0.1%,-1.5%,-0.9%
9,2015-03-31,-0.3%,-0.2%,-0.6%,-1.2%,-2.8%,-1.3%,0.3%,-1.7%,-0.5%
